In [4]:
import requests
import csv
import time
import json
from datetime import datetime, timedelta
import os
from pytz import UTC  # Import UTC timezone object

# Function to get the last date in the existing CSV file
def get_last_date_in_csv(file_path):
    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as f:
            csv_reader = csv.reader(f)
            last_row = None
            for row in csv_reader:
                last_row = row
            if last_row:
                return datetime.fromisoformat(last_row[5].replace("Z", "+00:00"))
    except FileNotFoundError:
        return None

# Initialize CSV file
csv_file_path = 'boston_311_data_2022.csv'
start_date = get_last_date_in_csv(csv_file_path) or datetime(2022, 1, 1, tzinfo=UTC)
end_date = datetime(2023, 1, 1, tzinfo=UTC)
delta = timedelta(days=1)  # 12 hours
minutedelta = timedelta(minutes=1)  # 1 minute

csv_file = open(csv_file_path, 'a', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Write header only if the file is new
if start_date == datetime(2023, 1, 1, tzinfo=UTC):
    csv_writer.writerow(['service_request_id', 'status', 'service_name', 'service_code', 'description', 'requested_datetime', 'updated_datetime', 'address', 'lat', 'long', 'token'])  # Header

while start_date <= end_date:
    formatted_start_date = start_date.strftime('%Y-%m-%dT%H:%M:%S') + 'Z'
    formatted_end_date = (start_date + delta).strftime('%Y-%m-%dT%H:%M:%S') + 'Z'

    url = f"https://311.boston.gov/open311/v2/requests.json?start_date={formatted_start_date}&end_date={formatted_end_date}"

    response = requests.get(url)
    print(f"Fetching data for {formatted_start_date} to {formatted_end_date}")
    #print number of requests in response
    print(f"Number of requests: {len(response.json())}")

    #print the wc -l of the file
    print(f"Number of lines in file: {os.system('wc -l boston_311_data_2022.csv')}")

    if response.status_code == 200:
        try:
            data = json.loads(response.text)
            for record in data:
                csv_writer.writerow([record.get('service_request_id'), record.get('status'), record.get('service_name'), record.get('service_code'), record.get('description'), record.get('requested_datetime'), record.get('updated_datetime'), record.get('address'), record.get('lat'), record.get('long'), record.get('token')])
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for {formatted_start_date} to {formatted_end_date}")
            continue  # Skip to the next iteration
    else:
        print(f"Failed to fetch data for {formatted_start_date} to {formatted_end_date}")

    # Move to next half-day
    #start_date += delta

    #set start_date to the last date in this response if it's greater than the current start_date else add delta. Use multiple lines
    if len(response.json()) > 0:
        last_date = datetime.fromisoformat(response.json()[-1].get('requested_datetime').replace("Z", "+00:00"))
        if last_date > start_date:
            start_date = last_date
        else:
            start_date += minutedelta


    # Rate limiting: Sleep for 6 seconds to stay within 10 requests per minute limit
    time.sleep(6)

# Close CSV file
csv_file.close()

Fetching data for 2022-01-15T11:23:00Z to 2022-01-16T11:23:00Z
Number of requests: 50
8542 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-15T14:31:00Z to 2022-01-16T14:31:00Z
Number of requests: 50
8582 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-15T17:19:00Z to 2022-01-16T17:19:00Z
Number of requests: 50
8620 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-15T19:10:00Z to 2022-01-16T19:10:00Z
Number of requests: 50
8687 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-15T21:47:00Z to 2022-01-16T21:47:00Z
Number of requests: 50
8735 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-15T23:28:00Z to 2022-01-16T23:28:00Z
Number of requests: 50
8779 boston_311_data_2022.csv
Number of lines in file: 0
Fetching data for 2022-01-16T01:12:00Z to 2022-01-17T01:12:00Z
Number of requests: 50
8823 boston_311_data_2022.csv
Number of lines in file: 0

In [2]:
#count the lines in the csv file
num_lines = sum(1 for line in open('boston_311_data_2022.csv'))

#print the count
print("Number of lines in the CSV file:")
print(num_lines)

Number of lines in the CSV file:
8532
